# 🔥 Fine-Tune CubiCasa Model with Fire Safety Detection

This notebook adds fire safety detection to your existing CubiCasa floor plan model.

**What it does:**
- Loads your trained CubiCasa model
- Fine-tunes it with VR_AI + safety fire safety datasets
- Creates one "Omega" model that detects BOTH floor plans + fire safety symbols

**Before running:**
1. Run `merge_datasets.py` locally
2. Zip the `fire_safety_merged` folder
3. Upload to your Google Drive

## 1️⃣ Setup

In [ ]:
# Install Ultralytics YOLOv8
!pip install -q ultralytics

from ultralytics import YOLO
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

print('✅ Setup complete!')

## 2️⃣ Upload Your CubiCasa Model

Upload your trained CubiCasa model (the `best.pt` file)

In [ ]:
from google.colab import files

print('📤 Upload your CubiCasa model (best.pt)...')
uploaded = files.upload()

# Rename to cubicasa_base.pt
for filename in uploaded.keys():
    os.rename(filename, 'cubicasa_base.pt')

print('✅ Model uploaded!')

## 3️⃣ Extract Fire Safety Dataset from Drive

**Update the path** to where you uploaded `fire_safety_merged.zip` in your Drive

In [ ]:
# UPDATE THIS PATH to your Google Drive location
DATASET_ZIP = '/content/drive/MyDrive/fire_safety_merged.zip'

# Extract dataset
!unzip -q {DATASET_ZIP} -d /content/

# Find the extracted folder
dataset_dir = '/content/fire_safety_merged'

print(f'✅ Dataset extracted to: {dataset_dir}')
print(f'📊 Contents:')
!ls -lh {dataset_dir}

## 4️⃣ Fine-Tune the Model

This will take **30-60 minutes** depending on dataset size.

In [ ]:
# Load your CubiCasa model as the base
model = YOLO('cubicasa_base.pt')

# Fine-tune with fire safety data
results = model.train(
    data=f'{dataset_dir}/data.yaml',
    epochs=50,              # Adjust if needed (50 is good for fine-tuning)
    imgsz=640,
    batch=16,
    device=0,               # Use GPU
    project='omega_training',
    name='cubicasa_firesafety',
    patience=10,            # Early stopping
    save=True,
    plots=True
)

print('\n🎉 Training complete!')
print('📁 Model saved to: /content/omega_training/cubicasa_firesafety/weights/best.pt')

## 5️⃣ Test the Model

In [ ]:
# Load the fine-tuned model
omega_model = YOLO('/content/omega_training/cubicasa_firesafety/weights/best.pt')

# Test on validation images
results = omega_model.val()

print('\n📊 Validation Results:')
print(f'mAP50: {results.box.map50:.3f}')
print(f'mAP50-95: {results.box.map:.3f}')

## 6️⃣ Download the Model

In [ ]:
from google.colab import files

# Download the trained model
model_path = '/content/omega_training/cubicasa_firesafety/weights/best.pt'
files.download(model_path)

print('✅ Model downloaded! Rename it to omega_model.pt')

## 7️⃣ (Optional) Convert to ONNX for Browser

If you want to use the model in your web app

In [ ]:
# Export to ONNX
omega_model.export(format='onnx', imgsz=640)

# Download ONNX model
onnx_path = '/content/omega_training/cubicasa_firesafety/weights/best.onnx'
files.download(onnx_path)

print('✅ ONNX model downloaded!')